# Using [SVC](https://github.com/rapidsai/cuml) with [RAPIDS](https://developer.nvidia.com/rapidshttps://developer.nvidia.com/rapids)

In [6]:
import pandas as pd
import cudf
from cuml import SVC
from cuml.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import time

df = cudf.read_csv('TV_News_Channel_Commercial_Detection_Dataset.csv')

# Convert cuDF DataFrame to pandas DataFrame for compatibility with other libraries
df_pandas = df.to_pandas()

In [7]:
# Separate the features (input variables) and the target variable (output variable)
X = df_pandas.drop('class', axis=1)
y = df_pandas['class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Normalize the data using cuML StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
c_values = [0.1, 1, 10, 100, 1000]

for c in c_values:
    svm = SVC(C=c, kernel='rbf', gamma='scale')
    
    start_time = time.time()
    
    svm.fit(X_train_scaled, y_train)
    
    elapsed_time = time.time() - start_time
    
    predictions = svm.predict(X_test_scaled)

    confusion_mat = confusion_matrix(y_test, predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions, average='binary')

    print("== [ C = {:.1f} ] ================".format(c))
    print("Confusion Matrix:")
    print(f"{confusion_mat}")
    print("F1 Score: {:.4f}".format(f1))
    print("Time spent: {:.4f} seconds\n".format(elapsed_time))


== [ C = 0.1 ] ================
Confusion Matrix:
[[1658  218]
 [ 335 1333]]
F1 Score: 0.8282
Time spent: 20.7222 seconds

== [ C = 1.0 ] ================
Confusion Matrix:
[[1744  132]
 [ 303 1365]]
F1 Score: 0.8626
Time spent: 22.6778 seconds

== [ C = 10.0 ] ================
Confusion Matrix:
[[1769  107]
 [ 305 1363]]
F1 Score: 0.8687
Time spent: 26.9014 seconds

== [ C = 100.0 ] ================
Confusion Matrix:
[[1756  120]
 [ 338 1330]]
F1 Score: 0.8531
Time spent: 30.1944 seconds

== [ C = 1000.0 ] ================
Confusion Matrix:
[[1750  126]
 [ 355 1313]]
F1 Score: 0.8452
Time spent: 41.6941 seconds

